Основные цели этого задания:

Попрактиковаться в борьбе с дисбалансом классов

Научиться заполнять пропуски в данных

Научиться использовать категориальные признаки.

Задача: по различным признакам, связанным с заявкой на грант, предсказать, будет ли заявка принята.

План решения:

Загрузите данные из csv файла. Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков.

Подготовьте данные к обучению моделей:

Отделите целевую переменную Grant.Status и выясните, сбалансированы ли классы. Если классы не сбалансированы, используйте в работе хотя бы один из изученных методов борьбы с дисбалансом классов;

Заполните пропуски

в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта),

в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;

Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;

Разделите данные на обучающую и тестовую части;

Используйте масштабирование для получения признаков одинакового масштаба.

Обучите модели и выберите лучшую:

Обучите модель логистической регрессии, используя кросс-валидацию. Оцените ее качество с помощью метрики rocauc. Выведите топ-10 признаков по важности, согласно обученной модели;

Обучите модель случайного леса

Для подбора гиперпараметров и кросс-валидации используйте структуру GridSearchCV,

Выберите наилучший вариант случайного леса и выведите его параметры,

Оцените качество выбранной модели с помощью метрики rocauc,

Выведите топ-10 признаков по важности. Используйте атрибут feature_importances_, чтобы узнать важность признаков в деревянных моделях.

Напишите вывод, в котором будет содержаться информация о том, какие признаки важны согласно обеим моделям, и какая модель оказалась наилучшей для решения поставленной задачи.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


In [3]:
# Шаг 1: Загрузка данных и первичный анализ
data = pd.read_csv('grant_data_imb.csv')
print(data.head())
print(data.info())
print(data['Grant.Status'].value_counts())


   Grant.Status Sponsor.Code Grant.Category.Code  \
0             0          97A                 30B   
1             0          36D                 10A   
2             0         317A                 30D   
3             0          62B                 10B   
4             0           1A                 10A   

  Contract.Value.Band...see.note.A  RFCD.Code.1  RFCD.Percentage.1  \
0                               A      321024.0               50.0   
1                               G      300201.0              100.0   
2                              NaN     321013.0              100.0   
3                               B      321103.0               30.0   
4                              NaN     270603.0               60.0   

   RFCD.Code.2  RFCD.Percentage.2  RFCD.Code.3  RFCD.Percentage.3  ...  \
0     321013.0               30.0     291502.0               20.0  ...   
1          0.0                0.0          0.0                0.0  ...   
2          0.0                0.0          0

Датасет состоит из 4113 строк и 39 столбцов.

Целевой признак - 'Grant.Status', который указывает, была ли заявка на грант принята (1) или нет (0).

Среди признаков есть категориальные (например, 'Sponsor.Code' и 'Grant.Category.Code') и числовые (например, 'RFCD.Code.1', 'RFCD.Percentage.1' и другие).

Есть признаки с пропущенными значениями, такие как 'Contract.Value.Band...see.note.A' и 'With.PHD.1'.

В столбце 'Grant.Status' классы не сбалансированы, так как большинство заявок были приняты (3259) по сравнению с отклоненными (854).

In [4]:
# Шаг 2: Подготовка данных

# Разделение на признаки (X) и целевую переменную (y)
X = data.drop(columns=['Grant.Status'])
y = data['Grant.Status']

# Проверка баланса классов
class_balance = y.value_counts(normalize=True)
print("Баланс классов:\n", class_balance)

Баланс классов:
 0    0.792366
1    0.207634
Name: Grant.Status, dtype: float64


In [5]:
# Шаг 3: Заполнение пропусков

# Создание преобразователей для числовых и категориальных признаков
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(exclude=[np.number]).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединение преобразованных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [6]:
# Шаг 4: Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Шаг 5: Построение моделей

# Логистическая регрессия
logistic_regression = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('classifier', LogisticRegression())])

logistic_regression.fit(X_train, y_train)
y_pred_lr = logistic_regression.predict(X_test)
roc_auc_lr = roc_auc_score(y_test, y_pred_lr)

# Важность признаков для логистической регрессии
coef_lr = logistic_regression.named_steps['classifier'].coef_
feature_names_lr = (numeric_features.tolist() +
                     list(logistic_regression.named_steps['preprocessor']
                          .named_transformers_['cat'].named_steps['onehot']
                          .get_feature_names_out(input_features=categorical_features)))


coef_lr = coef_lr.ravel()
feature_importance_lr = pd.Series(coef_lr, index=feature_names_lr)
top_features_lr = feature_importance_lr.abs().nlargest(10).index

print("Модель логистической регрессии ROC AUC:", roc_auc_lr)
print("Топ-10 признаков логистической регрессии:\n", top_features_lr)


Модель логистической регрессии ROC AUC: 0.6994294922045119
Топ-10 признаков логистической регрессии:
 Index(['Contract.Value.Band...see.note.A_J ', 'Sponsor.Code_75C',
       'Sponsor.Code_34B', 'Sponsor.Code_15C', 'Sponsor.Code_24D',
       'Sponsor.Code_62B', 'Sponsor.Code_163C', 'Sponsor.Code_6B',
       'Sponsor.Code_36D', 'Sponsor.Code_47C'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
# Случайный лес
param_grid_rf = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, None]
}

random_forest = Pipeline(steps=[('preprocessor', preprocessor),
                                ('classifier', RandomForestClassifier(random_state=42))])

grid_search_rf = GridSearchCV(random_forest, param_grid=param_grid_rf,
                              cv=5, scoring='roc_auc', n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

best_rf = grid_search_rf.best_estimator_
roc_auc_rf = grid_search_rf.best_score_
best_params_rf = grid_search_rf.best_params_
# Важность признаков для случайного леса
feature_importance_rf = best_rf.named_steps['classifier'].feature_importances_
top_features_rf = [feature_names_lr[i] for i in np.argsort(feature_importance_rf)[::-1][:10]]

print("Лучшие параметры случайного леса:", best_params_rf)
print("ROC AUC для случайного леса:", roc_auc_rf)
print("Топ-10 признаков случайного леса:\n", top_features_rf)


Лучшие параметры случайного леса: {'classifier__max_depth': 20, 'classifier__n_estimators': 300}
ROC AUC для случайного леса: 0.9011531988164305
Топ-10 признаков случайного леса:
 ['Number.of.Unsuccessful.Grant.1', 'Number.of.Successful.Grant.1', 'RFCD.Code.1', 'SEO.Code.1', 'Contract.Value.Band...see.note.A_A ', 'Dept.No..1', 'Person.ID.1', 'A.1', 'B.1', 'A..1']


In [10]:
# Шаг 6: Выводы
print("Наилучшей моделью для данной задачи оказался случайный лес с параметрами:")
print(best_params_rf)
print("ROC AUC для случайного леса:", roc_auc_rf)
print("Топ-10 признаков, важных для обеих моделей:\n", top_features_lr.intersection(top_features_rf))

Наилучшей моделью для данной задачи оказался случайный лес с параметрами:
{'classifier__max_depth': 20, 'classifier__n_estimators': 300}
ROC AUC для случайного леса: 0.9011531988164305
Топ-10 признаков, важных для обеих моделей:
 Index([], dtype='object')
